In [1]:
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
from google.cloud import storage
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

In [3]:
project_id = "looker-assignment-113356033"
dataset_id = "final_project_dataset"

In [4]:
crendentials = service_account.Credentials.from_service_account_file(r"C:\nccu\workspace\dataModel_finalProjrct\looker-assignment-113356033-4959799503ac.json")

In [5]:
bigquery_client = bigquery.Client(project=project_id, credentials=crendentials)
storage_client = storage.Client(credentials=crendentials, project=project_id)

In [7]:
# ============================
# 1. Data Loading
# ============================
Bonk_query = """
SELECT
    Currency Name,
    Currency Symbol,
    Amount,
    Sender,
    Receiver,
    Block Number,
    Block Time
FROM `looker-assignment-113356033.final_project_dataset.Bonk_transfer`
"""

Bonk_data = bigquery_client.query(Bonk_query).to_dataframe() 

FLOKI_query_template = """
SELECT
    `Currency Name`,
    `Currency Symbol`,
    Amount,
    Sender,
    Receiver,
    `Block Number`,
    `Block Time`
FROM `looker-assignment-113356033.final_project_dataset.FLOKI_transfer_part{part_number}`
"""

# 動態生成 UNION ALL 查詢
union_queries = []
for i in range(1, 11):  # 從 part1 到 part10
    union_queries.append(FLOKI_query_template.format(part_number=i))
    
full_query = " UNION ALL ".join(union_queries)

# 執行一次查詢將所有部分整合到 FLOKI_data
FLOKI_data = bigquery_client.query(full_query).to_dataframe()

Mog_query_template = """
SELECT
    `Currency Name`,
    `Currency Symbol`,
    Amount,
    Sender,
    Receiver,
    `Block Number`,
    `Block Time`
FROM `looker-assignment-113356033.final_project_dataset.Mog_transfer_part{part_number}`
"""

# 動態生成 UNION ALL 查詢
union_queries = []
for i in range(1, 11):  # 從 part1 到 part10
    union_queries.append(Mog_query_template.format(part_number=i))
    
full_query = " UNION ALL ".join(union_queries)

# 執行一次查詢將所有部分整合到 FLOKI_data
Mog_data = bigquery_client.query(full_query).to_dataframe()

Pepe_query_template = """
SELECT
    `Currency Name`,
    `Currency Symbol`,
    Amount,
    Sender,
    Receiver,
    `Block Number`,
    `Block Time`
FROM `looker-assignment-113356033.final_project_dataset.Pepe_transfer_part{part_number}`
"""

# 動態生成 UNION ALL 查詢
union_queries = []
for i in range(1, 11):  # 從 part1 到 part10
    union_queries.append(Pepe_query_template.format(part_number=i))
    
full_query = " UNION ALL ".join(union_queries)

# 執行一次查詢將所有部分整合到 FLOKI_data
Pepe_data = bigquery_client.query(full_query).to_dataframe()

Shib_query_template = """
SELECT
    `Currency Name`,
    `Currency Symbol`,
    Amount,
    Sender,
    Receiver,
    `Block Number`,
    `Block Time`
FROM `looker-assignment-113356033.final_project_dataset.Shib_transfer_part{part_number}`
"""

# 動態生成 UNION ALL 查詢
union_queries = []
for i in range(1, 11):  # 從 part1 到 part10
    union_queries.append(Shib_query_template.format(part_number=i))
    
full_query = " UNION ALL ".join(union_queries)

# 執行一次查詢將所有部分整合到 FLOKI_data
Shib_data = bigquery_client.query(full_query).to_dataframe()

# ============================
# 2. Data Preprocessing
# ============================

def calculate_balances(transactions_df):
    """
    Calculate net balances for each address based on sent and received transactions,
    grouped by year_month to provide snapshots over time.

    Args:
        transactions_df (pd.DataFrame): DataFrame containing 'Sender', 'Receiver',
                                        'Amount', and 'year_month' columns.

    Returns:
        pd.DataFrame: A DataFrame with each address's calculated balance per year_month.
    """
    # Ensure 'Amount' is numeric
    transactions_df['Amount'] = pd.to_numeric(transactions_df['Amount'], errors='coerce').fillna(0)

    # Ensure 'year_month' column exists
    if 'year_month' not in transactions_df.columns:
        transactions_df['year_month'] = transactions_df['Block Time'].dt.to_period('M').astype(str)

    # Calculate total sent amounts per year_month
    sent_df = (
        transactions_df
        .groupby(['year_month', 'Sender'])['Amount']
        .sum()
        .reset_index()
        .rename(columns={'Sender': 'Address', 'Amount': 'Sent'})
    )

    # Calculate total received amounts per year_month
    received_df = (
        transactions_df
        .groupby(['year_month', 'Receiver'])['Amount']
        .sum()
        .reset_index()
        .rename(columns={'Receiver': 'Address', 'Amount': 'Received'})
    )

    # Merge sent and received amounts
    balance_df = pd.merge(received_df, sent_df, on=['year_month', 'Address'], how='outer').fillna(0)

    # Calculate Net Balance
    balance_df['Balance'] = balance_df['Received'] - balance_df['Sent']

    # Sort the results for clarity
    balance_df = balance_df.sort_values(by=['year_month', 'Balance'], ascending=[True, False]).reset_index(drop=True)

    print(f"[INFO] Calculated balances snapshot for {balance_df['year_month'].nunique()} months and {len(balance_df)} address-month pairs.")
    return balance_df


def transform_block_time(df, time_column='Block Time'):
    """
    Transform 'Block Time' into datetime and extract year_month and year_month_day.

    Args:
        df (pd.DataFrame): DataFrame with a time column.
        time_column (str): Name of the time column.

    Returns:
        pd.DataFrame: Updated DataFrame with 'year_month' and 'year_month_day' columns.
    """
    try:
        df[time_column] = pd.to_datetime(df[time_column], errors='coerce')
        df['year_month'] = df[time_column].dt.to_period('M').astype(str)
        df['year_month_day'] = df[time_column].dt.strftime('%Y-%m-%d')
        print(f"[INFO] Transformed '{time_column}' into 'year_month' and 'year_month_day'.")
        return df
    except Exception as e:
        print(f"[ERROR] Failed to transform '{time_column}': {e}")
        return df

# ============================
# 3. Data Saving
# ============================
def save_balances(balance_df, file_path):
    """
    Save balance data to a CSV file.

    Args:
        balance_df (pd.DataFrame): DataFrame containing balance data.
        file_path (str): Path to save the CSV file.
    """
    try:
        balance_df.to_csv(file_path, index=False)
        print(f"[INFO] Saved balance data to {file_path}. Total rows: {len(balance_df)}")
    except Exception as e:
        print(f"[ERROR] An error occurred while saving the file: {e}")

# ============================
# . Main Function
# ============================
def main():
    # Example data
    data_names = [
        FLOKI_data,
        Bonk_data,
        Mog_data,
        Shib_data,
        Pepe_data
    ]

    directory_path = [
        'data/Floki',
        'data/Bonk',
        'data/Mog',
        'data/Shib',
        'data/Pepe'
    ]

    # Load and combine data
    for data, trans_path in zip(data_names, directory_path):
        # Calculate balances
        balance_df = calculate_balances(data)
        # Save balances
        save_balances(balance_df, f'transform_data/{trans_path[5:]}_token_balances.csv')

# ============================
# Entry Point
# ============================
if __name__ == '__main__':
    main()


[INFO] Found 13 files. Loading...
[INFO] Loaded data/Floki\FLOKI_transfers_part1.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part10.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part11.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part12.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part13.xlsx with 2240 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part2.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part3.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part4.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part5.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part6.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers_part7.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Floki\FLOKI_transfers

C:\Users\wise\AppData\Local\Temp\ipykernel_10064\2966295440.py:110: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df[time_column].dt.to_period('M').astype(str)


[INFO] Transformed 'Block Time' into 'year_month' and 'year_month_day'.
[INFO] Combined DataFrame shape: (1202240, 9)
[INFO] Calculated balances snapshot for 36 months and 335829 address-month pairs.
[INFO] Saved balance data to transform_data/Floki_token_balances.csv. Total rows: 335829
[INFO] Found 1 files. Loading...
[INFO] Loaded data/Bonk\Bonk_transfers.xlsx with 166378 rows and 7 columns.


C:\Users\wise\AppData\Local\Temp\ipykernel_10064\2966295440.py:110: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df[time_column].dt.to_period('M').astype(str)


[INFO] Transformed 'Block Time' into 'year_month' and 'year_month_day'.
[INFO] Combined DataFrame shape: (166378, 9)
[INFO] Calculated balances snapshot for 23 months and 47745 address-month pairs.
[INFO] Saved balance data to transform_data/Bonk_token_balances.csv. Total rows: 47745
[INFO] Found 20 files. Loading...
[INFO] Loaded data/Mog\Mog_transfers_part1.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part10.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part11.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part12.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part13.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part14.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part15.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfers_part16.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Mog\Mog_transfe

...

[INFO] Transformed 'Block Time' into 'year_month' and 'year_month_day'.
[INFO] Combined DataFrame shape: (1932980, 9)
[INFO] Calculated balances snapshot for 18 months and 182594 address-month pairs.
[INFO] Saved balance data to transform_data/Mog_token_balances.csv. Total rows: 182594
[INFO] Found 148 files. Loading...
[INFO] Loaded data/Shib\SHIB_transfers_part1.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part10.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part100.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part101.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part102.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part103.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part104.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Shib\SHIB_transfers_part105.xlsx with 100000 rows and 7 columns.
[INFO] Lo

C:\Users\wise\AppData\Local\Temp\ipykernel_10064\2966295440.py:110: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df[time_column].dt.to_period('M').astype(str)


[INFO] Transformed 'Block Time' into 'year_month' and 'year_month_day'.
[INFO] Combined DataFrame shape: (14773341, 9)
[INFO] Calculated balances snapshot for 54 months and 7520148 address-month pairs.
[INFO] Saved balance data to transform_data/Shib_token_balances.csv. Total rows: 7520148
[INFO] Found 52 files. Loading...
[INFO] Loaded data/Pepe\PEPE_transfers_part1.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part10.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part11.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part12.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part13.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part14.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part15.xlsx with 100000 rows and 7 columns.
[INFO] Loaded data/Pepe\PEPE_transfers_part16.xlsx with 100000 rows and 7 columns.
[INFO] Loade

C:\Users\wise\AppData\Local\Temp\ipykernel_10064\2966295440.py:110: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df[time_column].dt.to_period('M').astype(str)


[INFO] Transformed 'Block Time' into 'year_month' and 'year_month_day'.
[INFO] Combined DataFrame shape: (5178110, 9)
[INFO] Calculated balances snapshot for 21 months and 1475375 address-month pairs.
[INFO] Saved balance data to transform_data/Pepe_token_balances.csv. Total rows: 1475375


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import os
import numpy as np

# ============================
# 1. Load Balance Data
# ============================
def load_balance_data(file_path):
    """
    Load and preprocess balance data from a CSV or Excel file.

    Args:
        file_path (str): Path to the balance data file.

    Returns:
        pd.DataFrame: Cleaned and preprocessed DataFrame with 'year_month', 'Address', and 'Balance' columns.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"[ERROR] The file '{file_path}' does not exist.")

    try:
        # Load data based on file extension
        if file_path.endswith('.csv'):
            balance_df = pd.read_csv(file_path)
        elif file_path.endswith(('.xls', '.xlsx')):
            balance_df = pd.read_excel(file_path)
        else:
            raise ValueError("[ERROR] Unsupported file format. Use CSV or Excel files.")

        print(f"[INFO] Loaded data from {file_path}: {balance_df.shape[0]} rows, {balance_df.shape[1]} columns.")

        # Ensure required columns exist
        required_columns = {'year_month', 'Address', 'Balance'}
        if not required_columns.issubset(balance_df.columns):
            raise ValueError(f"[ERROR] File must contain the following columns: {required_columns}")

        # Ensure 'year_month' is a string
        balance_df['year_month'] = balance_df['year_month'].astype(str)

        # Ensure 'Balance' is numeric
        balance_df['Balance'] = pd.to_numeric(balance_df['Balance'], errors='coerce').fillna(0)

        # Remove invalid or zero balances
        balance_df = balance_df[balance_df['Balance'] > 0]

        # Sort data for clarity
        balance_df = balance_df.sort_values(by=['year_month', 'Balance'], ascending=[True, False]).reset_index(drop=True)

        print(f"[INFO] Preprocessed data: {balance_df.shape[0]} valid rows remaining.")
        return balance_df

    except Exception as e:
        print(f"[ERROR] Failed to load and preprocess data: {e}")
        return pd.DataFrame()

# ============================
# 2. Plot Token Distribution by Year-Month
# ============================
def plot_all_token_snapshots(balance_df):
    """
    Plot token balance distributions for all year_month snapshots in one plot.

    Args:
        balance_df (pd.DataFrame): DataFrame with 'year_month', 'Address', and 'Balance'.
    """
    # Ensure required columns exist
    required_columns = {'year_month', 'Address', 'Balance'}
    if not required_columns.issubset(balance_df.columns):
        raise ValueError(f"[ERROR] DataFrame must contain the following columns: {required_columns}")

    # Get unique year_month values
    available_months = sorted(balance_df['year_month'].unique())

    # Define color palette
    colors = plt.cm.tab20(np.linspace(0, 1, len(available_months)))

    plt.figure(figsize=(14, 8))

    for i, month in enumerate(available_months):
        filtered_df = balance_df[balance_df['year_month'] == month]

        if filtered_df.empty:
            print(f"[WARNING] No data available for {month}. Skipping...")
            continue

        # Plot histogram for each month
        plt.hist(
            filtered_df['Balance'],
            bins=50,
            alpha=0.5,
            label=f'{month}',
            color=colors[i],
            log=True
        )

    plt.title('Token Balance Distribution Across Year-Month Snapshots')
    plt.xlabel('Token Balance (Log Scale)')
    plt.ylabel('Number of Addresses')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0), fontsize='small')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()

    save_plot('Balance long tail(year_month)', 'Combined_token')
    plt.show()

def plot_half_year_token_snapshots(balance_df):
    """
    Plot token balance distributions for half-year snapshots in one plot.

    Args:
        balance_df (pd.DataFrame): DataFrame with 'year_month', 'Address', and 'Balance'.
    """
    # Ensure required columns exist
    required_columns = {'year_month', 'Address', 'Balance'}
    if not required_columns.issubset(balance_df.columns):
        raise ValueError(f"[ERROR] DataFrame must contain the following columns: {required_columns}")

    # Ensure 'year_month' is datetime for easier manipulation
    balance_df['year_month'] = pd.to_datetime(balance_df['year_month'], format='%Y-%m')

    # Create half-year column
    balance_df['half_year'] = balance_df['year_month'].dt.year.astype(str) + \
                              '-H' + ((balance_df['year_month'].dt.month - 1) // 6 + 1).astype(str)

    # Get unique half-year values
    available_half_years = sorted(balance_df['half_year'].unique())

    # Define color palette
    colors = plt.cm.tab20(np.linspace(0, 1, len(available_half_years)))

    plt.figure(figsize=(14, 8))

    for i, half_year in enumerate(available_half_years):
        filtered_df = balance_df[balance_df['half_year'] == half_year]

        if filtered_df.empty:
            print(f"[WARNING] No data available for {half_year}. Skipping...")
            continue

        # Plot histogram for each half-year
        plt.hist(
            filtered_df['Balance'],
            bins=50,
            alpha=0.5,
            label=f'{half_year}',
            color=colors[i],
            log=True
        )

    plt.title('Token Balance Distribution Across Half-Year Snapshots')
    plt.xlabel('Token Balance (Log Scale)')
    plt.ylabel('Number of Addresses')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0), fontsize='small')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()

    save_plot('Balance long tail(half_year)', 'Combined_token')
    plt.show()


def plot_yearly_token_snapshots(balance_df):
    """
    Plot token balance distributions for yearly snapshots in one plot.

    Args:
        balance_df (pd.DataFrame): DataFrame with 'year_month', 'Address', and 'Balance'.
    """
    # Ensure required columns exist
    required_columns = {'year_month', 'Address', 'Balance'}
    if not required_columns.issubset(balance_df.columns):
        raise ValueError(f"[ERROR] DataFrame must contain the following columns: {required_columns}")

    # Ensure 'year_month' is datetime for easier manipulation
    balance_df['year_month'] = pd.to_datetime(balance_df['year_month'], format='%Y-%m')

    # Create a 'year' column
    balance_df['year'] = balance_df['year_month'].dt.year.astype(str)

    # Get unique years
    available_years = sorted(balance_df['year'].unique())

    # Define color palette
    colors = plt.cm.tab20(np.linspace(0, 1, len(available_years)))

    # Plot setup
    plt.figure(figsize=(14, 8))

    for i, year in enumerate(available_years):
        filtered_df = balance_df[balance_df['year'] == year]

        if filtered_df.empty:
            print(f"[WARNING] No data available for {year}. Skipping...")
            continue

        # Plot histogram for each year
        plt.hist(
            filtered_df['Balance'],
            bins=50,
            alpha=0.5,
            label=f'{year}',
            color=colors[i],
            log=True
        )

    # 📊 Titles and Labels
    plt.title('Token Balance Distribution Across Yearly Snapshots')
    plt.xlabel('Token Balance (Log Scale)')
    plt.ylabel('Number of Addresses')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0), fontsize='small')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()

    save_plot('Balance long tail(year)', 'Combined_token')
    # Show Plot
    plt.show()

# ============================
# 3. Save Plots
# ============================
def save_plot(file_name, plot_type, output_dir='plots'):
    """
    Save the current plot to a specified directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plot_file_name = f"{file_name}_{plot_type}.png"
    plot_path = os.path.join(output_dir, plot_file_name)
    plt.savefig(plot_path, bbox_inches='tight')
    print(f"[INFO] Plot saved: {plot_path}")
    plt.close()

# ============================
# . Main Function
# ============================
def main():
   # Define file paths and patterns
    directory_path = 'transform_data'
    file_names = [
        'Floki_token_balances.csv',
        'Bonk_token_balances.csv',
        'Mog_token_balances.csv',
        'Shib_token_balances.csv',
        'Pepe_token_balances.csv'
    ]
    token_names = ['FLOKI', 'BONK', 'MOG', 'SHIB', 'PEPE']
    #token_names = ['FLOKI', 'BONK', 'MOG', 'PEPE']

    # Plot Token Distribution
    if not balance_df.empty:
        plot_all_token_snapshots(balance_df)
        plot_half_year_token_snapshots(balance_df)
        plot_yearly_token_snapshots(balance_df)
    else:
        print("[ERROR] No data available for plotting.")

# ============================
# Entry Point
# ============================
if __name__ == '__main__':
    main()


[INFO] Loaded data from transform_data/Flokitoken_balances.csv: 335829 rows, 5 columns.
[INFO] Preprocessed data: 180250 valid rows remaining.
[INFO] Plot saved: plots\Balance long tail(year_month)_Combined_token.png
[INFO] Plot saved: plots\Balance long tail(half_year)_Combined_token.png
[INFO] Plot saved: plots\Balance long tail(year)_Combined_token.png
